In [4]:
# BERT practice
# kwanki.ahn 2021/01/23(Sat)
# ref
#   https://www.notion.so/kwankiahn/BERT-Classification-by-PyTorch-d4eb04caf6354058802b8aad3494c53e
#   https://medium.com/@eyfydsyd97/bert%EB%A5%BC-%ED%99%9C%EC%9A%A9%ED%95%9C-classification-by-pytorch-2a6d4adaf162
#   https://www.kaggle.com/abhishek/bert-base-uncased-using-pytorch

#   https://medium.com/@eyfydsyd97/bert%EB%A5%BC-%ED%99%9C%EC%9A%A9%ED%95%9C-classification-by-pytorch-2a6d4adaf162

In [6]:
import tokenizers

In [7]:
max_len = 112 # No tweet is longer than 108
train_batch_size = 64
valid_batch_size = 16
epochs = 5
bert_path = './data/bert-base-uncased/vocab.txt'
modle_path = "bertbaseuncased.bin"
train_file = "./data/tweet-sentiment-extraction/train.csv"
test_file = "./data/tweet-sentiment-extraction/test.csv"
tokenizer = tokenizers.BertWordPieceTokenizer(bert_path,lowercase=True)

In [10]:
data = pd.read_csv(train_file)
data[:10]

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative
5,28b57f3990,http://www.dothebouncy.com/smf - some shameles...,http://www.dothebouncy.com/smf - some shameles...,neutral
6,6e0c6d75b1,2am feedings for the baby are fun when he is a...,fun,positive
7,50e14c0bb8,Soooo high,Soooo high,neutral
8,e050245fbd,Both of you,Both of you,neutral
9,fc2cbefa9d,Journey!? Wow... u just became cooler. hehe....,Wow... u just became cooler.,positive


# 2. Data Preprocessing(데이터 전처리)

In [12]:
# train valid 분리하기
train = data[:24000]
valid = data[24000:]

train = train.dropna(axis=0)
train = train.reset_index()
valid = valid.dropna(axis=0)
valid = valid.reset_index()
train_text = train['text']
train_label = train['sentiment']
valid_text = valid['text']
valid_label = valid['sentiment']

#label 숫자로 바꾸기
def label_convert(label):
    for i in range(len(label)):
        if label[i] == 'positive':
            label[i] = 0
        if label[i] == 'neutral':
            label[i] = 1
        if label[i] == 'negative':
            label[i] = 2
    return label    
train_label=label_convert(train_label)
valid_label=label_convert(valid_label)  

/home/kwankiahn/setup/venv-3/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/kwankiahn/setup/venv-3/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/kwankiahn/setup/venv-3/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


우리가 BERT에 넣어야 할 데이터는 다음과 같다.

1) Input_ids (Tokenizer된 데이터)

2) Token_type_ids (문장 유형을 구분한다.)

3) Attention_mask_ids (masking을 할 수 있게 해준다)

In [22]:
import torch
import transformers
from transformers import BertTokenizer

tokenizer= BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)

def text_processing(text, Max_len=30):
    input_ids = []
    mask_ids = []
    token_type_ids = []
    for line in text:
        # [cls], [sep] 추가 Max_len 길이 맞추기  
        encoded_line = tokenizer.encode(line, add_special_tokens = True, max_length=Max_len, truncation=True) 
        input = encoded_line + [0]*(Max_len-len(encoded_line))
        token_type =  [0]*len(input)
        mask = [1]*len(encoded_line)+[0]*(Max_len-len(encoded_line))    
        input_ids.append(torch.tensor(input))
        token_type_ids.append(torch.tensor(token_type))
        mask_ids.append(torch.tensor(mask))

    return input_ids, token_type_ids, mask_ids
  
train_input_ids, train_token_type_ids, train_mask_ids =text_processing(train_text)
valid_input_ids, valid_token_type_ids, valid_mask_ids =text_processing(valid_text)  

# 3. Dataset & Dataloader
그 다음 Custom Dataset을 만들고 그 후 Dataloader을 만들어준다.

In [24]:
Batch_size = 1024

In [25]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class BertCls_dataset(Dataset):
  def __init__(self, input_ids, token_type_ids, mask_ids, label):
    self.input = input_ids
    self.token_type = token_type_ids
    self.attn_mask = mask_ids
    self.label = label
  def __len__(self):
    return len(self.input)  

  def __getitem__(self, idx):
    x = self.input[idx]
    y = self.token_type[idx]
    z = self.attn_mask[idx]
    label = self.label[idx]
    label = [label]

    return x, y, z ,torch.tensor(label)

train_dataset=BertCls_dataset(train_input_ids, train_token_type_ids, train_mask_ids, train_label)
valid_dataset=BertCls_dataset(valid_input_ids, valid_token_type_ids, valid_mask_ids, valid_label)

train_dataloader = DataLoader(train_dataset, batch_size=Batch_size, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=Batch_size, shuffle=True)

# 4. Model

In [26]:
import torch
import torch.nn as nn
from transformers import BertModel

bertcls = BertModel.from_pretrained("bert-base-uncased")

class BertCls(nn.Module):
  def __init__(self):
    super().__init__()
    self.bert_cls = bertcls
    self.dropout = nn.Dropout(0.3)
    self.relu = nn.ReLU()
    self.out = nn.Linear(768*2, num_category)

  def forward(self, input_ids, token_type_ids, mask_ids):
    output,_ = self.bert_cls(input_ids, attention_mask=mask_ids, token_type_ids=token_type_ids) #output은 hidden_state
    apool = torch.mean(output, 1)
    mpool, _ = torch.max(output, 1)
    x = torch.cat((apool, mpool), 1) #[cls]를 사용하는 방식 대신, mean, max를 한 후 concat한 output을 사용하였다.
    x = self.relu(x)
    x = self.dropout(x)
    x = self.out(x)

    return x

# 5. Train & eval

Train에 대한 코드는 아래를 살펴보자

In [27]:
def train_one_epoch(data_loader, model, optimizer, device, loss_fn):
  
  model.train()
  tk0 = tqdm(data_loader, total=len(data_loader))
  total_loss = 0.0
  
  for bi, d in enumerate(tk0):
      input_ids, token_type_ids, attn_mask_ids, label = d
      input_ids = input_ids.to(device, dtype=torch.long)
      token_type_ids = token_type_ids.to(device, dtype=torch.long)
      attn_mask_ids = attn_mask_ids.to(device, dtype=torch.long)
      label = label.to(device, dtype=torch.long)

      #model.zero_grad()
      output = model(input_ids, token_type_ids, attn_mask_ids)
      loss = loss_fn(output, label.view(-1))
      total_loss += loss.item()
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

  avg_train_loss = total_loss / len(data_loader) 
  print(" Average training loss: {0:.2f}".format(avg_train_loss))  

def eval_one_epoch(data_loader, model,  device, loss_fn):

  model.eval()
  tk0 = tqdm(data_loader, total=len(data_loader))
  fin_targets = []
  fin_outputs = []
  
  with torch.no_grad():

    for bi, d in enumerate(tk0):
      input_ids, token_type_ids, attn_mask_ids, label = d
      input_ids = input_ids.to(device, dtype=torch.long)
      token_type_ids = token_type_ids.to(device, dtype=torch.long)
      attn_mask_ids = attn_mask_ids.to(device, dtype=torch.long)
      label = label.to(device, dtype=torch.long)

      output = model(input_ids, token_type_ids, attn_mask_ids)
      loss = loss_fn(output, label.view(-1))

      output = output.detach().cpu().numpy()
      label = label.detach().cpu().numpy()
      pred = np.argmax(output, axis=1).flatten()

      fin_targets.extend(label.tolist())
      fin_outputs.extend(pred.tolist()) 

    
  return fin_outputs, fin_targets
  
def fit(train_dataloader, valid_dataloader, EPOCHS=3):
  Bert_cls=BertCls() #model
  Bert_cls=Bert_cls.to(device)
  loss_fn = nn.CrossEntropyLoss() #loss
  optimizer = torch.optim.AdamW(Bert_cls.parameters(),lr=lr) #optimizer

  for i in range(EPOCHS):
    print(f"EPOCHS:{i+1}")
    print('TRAIN')
    train_one_epoch(train_dataloader, Bert_cls, optimizer, device, loss_fn)
    print('EVAL')
    outputs, targets = eval_one_epoch(valid_dataloader, Bert_cls,  device, loss_fn)
    targets = np.array(targets)
    auc = accuracy_score(targets, outputs)
    print(f"auc;{auc}")  
    
    
Max_len = 64
Batch_size = 16
num_category = 3
lr = 3e-5
device='cuda'
EPOCHS=3

fit(train_dataloader, valid_dataloader)    

RuntimeError: cuda runtime error (100) : no CUDA-capable device is detected at /pytorch/aten/src/THC/THCGeneral.cpp:47